In [1]:
import pandas as pd
import numpy as np

In [4]:
df_raw1=pd.read_csv('Data/food_choices.csv')

In [6]:
df_raw1

,Unnamed: 0,cook,diet_current_coded,eating_out,sports,exercise,fav_cuisine_coded,on_off_campus,pay_meal_out,fav_food,fruit_day,income
0,0,2,1,3,1,1,3,1,2,1,5,5
1,1,3,2,2,1,1,1,1,4,1,4,4
2,2,1,3,2,2,2,1,2,3,3,5,6
3,3,2,2,2,2,3,3,1,2,1,4,6
4,4,1,2,2,1,1,1,1,4,3,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
120,120,3,2,2,1,2,1,3,4,1,5,4
121,121,3,2,4,1,2,2,1,4,1,4,2
122,122,3,2,3,2,2,4,1,4,1,4,2
123,123,3,1,5,2,1,1,1,3,3,5,4


In [8]:
df_raw2=pd.read_csv('Data/food_coded.csv')

In [27]:
df_raw2.columns

Index(['GPA', 'Gender', 'breakfast', 'calories_chicken', 'calories_day',
       'calories_scone', 'coffee', 'comfort_food', 'comfort_food_reasons',
       'comfort_food_reasons_coded', 'cook', 'comfort_food_reasons_coded.1',
       'cuisine', 'diet_current', 'diet_current_coded', 'drink',
       'eating_changes', 'eating_changes_coded', 'eating_changes_coded1',
       'eating_out', 'employment', 'ethnic_food', 'exercise',
       'father_education', 'father_profession', 'fav_cuisine',
       'fav_cuisine_coded', 'fav_food', 'food_childhood', 'fries', 'fruit_day',
       'grade_level', 'greek_food', 'healthy_feeling', 'healthy_meal',
       'ideal_diet', 'ideal_diet_coded', 'income', 'indian_food',
       'italian_food', 'life_rewarding', 'marital_status',
       'meals_dinner_friend', 'mother_education', 'mother_profession',
       'nutritional_check', 'on_off_campus', 'parents_cook', 'pay_meal_out',
       'persian_food', 'self_perception_weight', 'soup', 'sports', 'thai_food',
       

In [10]:
df_raw3=pd.read_csv('Data/food_cleaned_data.csv')

In [29]:
df_raw3

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight,Spending Score (1-100)
0,2.400,2,1,430,3,315,1,none,we dont have comfort,9,...,1,1,1165,345,car racing,5,1,1315,187,39
1,3.654,1,1,610,3,420,2,"chocolate, chips, ice cream","Stress, bored, anger",1,...,1,2,725,690,Basketball,4,2,900,155,81
2,3.300,1,1,720,4,420,2,"frozen yogurt, pizza, fast food","stress, sadness",1,...,2,5,1165,500,none,5,1,900,158,6
3,3.200,1,1,430,3,420,2,"Pizza, Mac and cheese, ice cream",Boredom,2,...,2,5,725,690,NaN,3,1,1315,158,77
4,3.500,1,1,720,2,420,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1,...,1,4,940,500,Softball,4,2,760,190,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,3.500,1,1,610,4,420,2,"wine. mac and cheese, pizza, ice cream",boredom and sadness,3,...,1,5,940,500,Softball,5,1,1315,156,56
121,3.000,1,1,265,2,315,2,Pizza / Wings / Cheesecake,Loneliness / Homesick / Sadness,3,...,1,4,940,500,basketball,5,2,1315,180,40
122,3.882,1,1,720,3,420,1,"rice, potato, seaweed soup",sadness,3,...,2,5,580,690,none,4,2,1315,120,58
123,3.000,2,1,720,4,420,1,"Mac n Cheese, Lasagna, Pizza","happiness, they are some of my favorite foods",3,...,2,1,940,500,NaN,3,1,1315,135,91


In [12]:
df_raw3['calories_chicken'].unique()

array([430, 610, 720, 265])